In [1]:
%%time
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
CPU times: user 2.11 s, sys: 613 ms, total: 2.73 s
Wall time: 3min 23s


In [2]:
!pip install transformers peft accelerate \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

Looking in links: /kaggle/input/lmsys-wheel-files
ERROR: Could not find a version that satisfies the requirement peft (from versions: none)
ERROR: No matching distribution found for peft


In [3]:
import os, math, numpy as np
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
pd.set_option('display.max_rows', 300)

# Evaluation Metrics

In [4]:
%%writefile eedi_metrics.py

# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

Writing eedi_metrics.py


# Data

In [5]:
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)

df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(1, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

In [6]:
INFERENCE = True

# Retrieval

In [7]:
# Testing 
if INFERENCE:
    testing_code = df_test.copy()
else:
    testing_code = df_train.copy()

testing_code.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct


In [8]:
from sentence_transformers import SentenceTransformer, util

# embedding text
model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')

In [9]:
# I see implementations that use cosine similarity to generate choices for the llm to choose from
# Rather than use a specific problem, we can use general terms 
# Subject and construct are consistent across all of the entries (as in they are categories)

def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub("@\w+", '',x)      # Delete strings starting with @
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

testing_code['Input'] = testing_code['ConstructName'] + '; ' + testing_code['SubjectName']

# since this is a string, we can preprocess it
testing_code['Input'] = testing_code['Input'].apply(lambda x: preprocess_text(x))

# to compute cosine sim, we need to vectorize the sentences and misconceptions
embedding_query = model.encode(testing_code['Input'], convert_to_tensor=True)
misconceptions = df_misconception_mapping.MisconceptionName.values
embedding_Misconception = model.encode(misconceptions, convert_to_tensor=True)

# util is a library part of sentence transformers that has useful utilities
# this computes cos sim between the two vectors
topk_ids = util.semantic_search(embedding_query, embedding_Misconception, top_k=150)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [10]:
topk_ids[0]

[{'corpus_id': 1672, 'score': 0.8738340139389038},
 {'corpus_id': 1941, 'score': 0.8647750616073608},
 {'corpus_id': 987, 'score': 0.8582115769386292},
 {'corpus_id': 2488, 'score': 0.8519265055656433},
 {'corpus_id': 2518, 'score': 0.8426055908203125},
 {'corpus_id': 2532, 'score': 0.8379297852516174},
 {'corpus_id': 328, 'score': 0.8340588808059692},
 {'corpus_id': 1516, 'score': 0.828125},
 {'corpus_id': 706, 'score': 0.8089470863342285},
 {'corpus_id': 1507, 'score': 0.8044964075088501},
 {'corpus_id': 2221, 'score': 0.8039247989654541},
 {'corpus_id': 158, 'score': 0.7999632358551025},
 {'corpus_id': 1054, 'score': 0.7870170474052429},
 {'corpus_id': 1999, 'score': 0.7829920053482056},
 {'corpus_id': 15, 'score': 0.7828851342201233},
 {'corpus_id': 2087, 'score': 0.7817094922065735},
 {'corpus_id': 2441, 'score': 0.7763828039169312},
 {'corpus_id': 1005, 'score': 0.771702766418457},
 {'corpus_id': 2181, 'score': 0.7672696113586426},
 {'corpus_id': 77, 'score': 0.7627321481704712},

In [11]:
misconceptions

array(['Does not know that angles in a triangle sum to 180 degrees',
       'Uses dividing fractions method for multiplying fractions',
       'Believes there are 100 degrees in a full turn', ...,
       'Believes that any percentage of a larger number will be greater than any percentage of a smaller number',
       'Believes a cubic expression should have three terms',
       'Misunderstands order of operations in algebraic expressions'],
      dtype=object)

In [12]:
for idx, row in testing_code.iterrows():
    # this is the top k misconceptions for this row based on cos sim
    mis_ids = topk_ids[idx]
    mis_text = ''
    for mis_idx, ids in enumerate(mis_ids):
        mis_text += f"{mis_idx+1}: {misconceptions[ids['corpus_id']]} \n"
    testing_code.loc[idx, 'miscon'] = mis_text


testing_code.head()

    

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,Input,miscon
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,use the order of operations to carry out calcu...,"1: Confuses the order of operations, believes ..."
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,simplify an algebraic fraction by factorising ...,1: Thinks can combine the numerator and denomi...
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,calculate the range from a list of data; range...,1: When calculating the range does not reorder...


In [13]:
PROMPT  = """
You are a Mathematics tutor. Your task is to identify common misconceptions for the Incorrect Answer given the following Question.

The question topic is the following: {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

Answer which misconception is the reason for obtaining the incorrect answer.
Providing reasoning is not required.
Listed below is a list of common misconceptions that you can use to help make a decision:

{Retrieval}
"""

def apply_template(row, tokenizer, targetCol):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"Answer{targetCol}Text"],
                    CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"],
                    Retrieval=row[f"miscon"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

miscon_df = pd.DataFrame(columns=['QuestionId_Answer', 'prompt_text'])

if INFERENCE: 
    for idx, row in testing_code.iterrows():
        for answer in ['A', 'B', 'C', 'D']:
            if row['CorrectAnswer'] != answer:
                tmp_dict = {'QuestionId_Answer': f"{row['QuestionId']}_{answer}", 'prompt_text':apply_template(row, tokenizer, answer)}
                tmp_df = pd.DataFrame(tmp_dict, index=[0])
                miscon_df = pd.concat([miscon_df, tmp_df])
else:
    for idx, row in testing_code.iterrows():
        for answer in ['A', 'B', 'C', 'D']:
            if (row['CorrectAnswer']!=answer) and (row[f"Misconception{answer}Id"]!= -1):
                tmp_dict = {'QuestionId_Answer': f"{row['QuestionId']}_{answer}", 'prompt_text':apply_template(row, tokenizer, answer)}
                tmp_df = pd.DataFrame(tmp_dict, index=[0])
                miscon_df = pd.concat([miscon_df, tmp_df])

display(miscon_df.head())
miscon_df.to_parquet("miscon_df.parquet", index=False)
            

,QuestionId_Answer,prompt_text
0,1869_B,"<|im_start|>system\nYou are Qwen, created by A..."
0,1869_C,"<|im_start|>system\nYou are Qwen, created by A..."
0,1869_D,"<|im_start|>system\nYou are Qwen, created by A..."
0,1870_A,"<|im_start|>system\nYou are Qwen, created by A..."
0,1870_B,"<|im_start|>system\nYou are Qwen, created by A..."


In [14]:
# trying a lora for for inference?
"""
from peft import (
    LoraConfig,
    get_peft_model,
)

model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
device='cuda:0'
#bnb_config = BitsAndBytesConfig(
#            load_in_4bit=True,
#            bnb_4bit_use_double_quant=True,
#            bnb_4bit_quant_type="nf4",
#            bnb_4bit_compute_dtype=torch.bfloat16
#        )
model = AutoModel.from_pretrained(model_path, 
                                  #quantization_config=bnb_config, 
                                  device_map=device,
                                  trust_remote_code=True)
config = LoraConfig(
    r=32,
    lora_alpha=64,
        target_modules=[
            "q_proj",
            "k_proj",
  #          "v_proj",
  #          "o_proj",
  #          "up_proj",
  #          "down_proj",
        ],
    bias="none",
    lora_dropout = 0.05,
    task_type="FEATURE_EXTRACTION"
)

model = get_peft_model(model, config)

model.save_pretrained()
#torch.save(model, 'lora.pt')
"""

'\nfrom peft import (\n    LoraConfig,\n    get_peft_model,\n)\n\nmodel_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"\ntokenizer = AutoTokenizer.from_pretrained(model_path)\ndevice=\'cuda:0\'\n#bnb_config = BitsAndBytesConfig(\n#            load_in_4bit=True,\n#            bnb_4bit_use_double_quant=True,\n#            bnb_4bit_quant_type="nf4",\n#            bnb_4bit_compute_dtype=torch.bfloat16\n#        )\nmodel = AutoModel.from_pretrained(model_path, \n                                  #quantization_config=bnb_config, \n                                  device_map=device,\n                                  trust_remote_code=True)\nconfig = LoraConfig(\n    r=32,\n    lora_alpha=64,\n        target_modules=[\n            "q_proj",\n            "k_proj",\n  #          "v_proj",\n  #          "o_proj",\n  #          "up_proj",\n  #          "down_proj",\n        ],\n    bias="none",\n    lora_dropout = 0.05,\n    task_type="FEATURE_EXTRACTION"\n)\n\nmodel = get_peft_mo

# inference

In [15]:
%%writefile run_vllm.py
import re
import vllm
import pandas as pd

#df = pd.read_parquet("submission.parquet")
df = pd.read_parquet('miscon_df.parquet')

#model_path = "/kaggle/working/lora.pt"
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    df["prompt_text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.7,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=450,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)

responses = [x.outputs[0].text for x in responses]

def extract_response(text):
    return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()

df["llmMisconception"] = responses
df.to_csv('test.csv', index=False)
#df.to_parquet("submission.parquet", index=False)

Writing run_vllm.py


In [16]:
!python run_vllm.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 12-06 22:24:03 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-06 22:24:03 config.py:715] Defaulting to use mp for distributed inference
INFO 12-06 22:24:03 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig

In [17]:
#llm_output = pd.read_parquet("submission.parquet")
llm_output = pd.read_csv("test.csv")
for idx, row in llm_output.iterrows():
    print(row.llmMisconception)
    print("==="*6)

The misconception that leads to the incorrect answer \(3 \times 2+(4-5)\) is:

4: answers order of operations questions with brackets as if the brackets are not there

This misconception suggests that the student might have placed the brackets but did not actually change the order of operations according to the brackets, leading to an incorrect calculation.
The misconception that leads to the incorrect answer \(3 \times (2+4-5)\) is:

4: answers order of operations questions with brackets as if the brackets are not there

This misconception indicates that the student has placed the brackets but has not changed the order of operations accordingly, leading to an incorrect calculation.
The misconception that leads to the incorrect answer "does not need brackets" is:

6: believes order of operations does not affect the answer to a calculation

This misconception indicates that the student does not understand the necessity of using brackets to alter the order of operations to achieve a spec

In [18]:
llm_output

,QuestionId_Answer,prompt_text,llmMisconception
0,1869_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
1,1869_C,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
2,1869_D,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
3,1870_A,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
4,1870_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
5,1870_C,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
6,1871_A,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
7,1871_C,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...
8,1871_D,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...


In [19]:
text = llm_output.loc[0, 'prompt_text']
PREFIX = "<|im_start|>user"
text = text.split(PREFIX)[1].split("you are a Mathematics tutor.")[0].strip('\n').split('the question topic is the following:')[-1].strip()
print(text)

use the order of operations to carry out calculations involving powers(bidmas).
question: \[
3 \times 2+4-5
\]
where do the brackets need to go to make the answer equal 13 ?
correct answer: 3 \times(2+4)-5 
incorrect answer: 3 \times 2+(4-5) 

answer which misconception is the reason for obtaining the incorrect answer.
providing reasoning is not required.
listed below is a list of common misconceptions that you can use to help make a decision:

1: confuses the order of operations, believes addition comes before multiplication 
2: confuses the order of operations, believes subtraction comes before multiplication 
3: believes that the order of a worded calculation should be changed to follow bidmas 
4: answers order of operations questions with brackets as if the brackets are not there 
5: may have made a calculation error using the order of operations 
6: believes order of operations does not affect the answer to a calculation 
7: performs addition ahead of multiplication 
8: performs a

# post processing

In [20]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

#df = pd.read_parquet("submission.parquet")
df = pd.read_csv("test.csv")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')

In [21]:
def number2sentence(row):
    """
    This is used for post-processing of LLM's output.
    Since we give top-N retrieval to the LLM with serial number,
    Sometimes the LLM will only output the serial number without any sentence.
    We use the 'dicts' generated at the beginning to map the serial number with corresponding misconceptions.
    """
    text = row['llmMisconception'].strip()
    # potential is the most possible serial number in LLM output.
    potential = re.search(r'^\w+\.{0,1}', text).group()
    if '.' in potential:
        sentence = text.replace(potential, '').strip()
    # if the LLM output is only a serial number, we map it with corresponding misconceptions saved in the dict.
    elif len(potential) == len(text):
        qid_retrieval = dicts[row['QuestionId']]
        try:
            # qid_retrieval is the top-N misconceptions for an QuestionId,
            # qid_retrieval[potential] is the most possible misconception.
            sentence = qid_retrieval[potential]
        except:
            # If the mapping fails, we use the first one(the most possible one in the first retrieval).
            sentence = qid_retrieval['1']
    else:
        sentence = text
        
    return sentence


df['QuestionId'] = df['QuestionId_Answer'].apply(lambda x: x.split('_')[0])
df['llmMisconception_clean'] = df.apply(number2sentence, axis=1)

In [22]:
df.head(5)

,QuestionId_Answer,prompt_text,llmMisconception,QuestionId,llmMisconception_clean
0,1869_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...
1,1869_C,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...
2,1869_D,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...
3,1870_A,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1870,The misconception that leads to the incorrect ...
4,1870_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1870,The misconception that leads to the incorrect ...


# Second Retrieval

Uses the output of the first retrieval to obtain a misconception. Then uses that, along with other things like the subject and construct, for cosine similarity for the top 25

In [23]:
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub(r"@\w+", '',x)      # Delete strings starting with @
    #x = re.sub(r"\d+", '',x)      # Delete Numbers
    x = re.sub(r"http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PREFIX = "<|im_start|>user"
df['input_features'] = df["prompt_text"].apply(lambda x: x.split(PREFIX)[1].split("you are a Mathematics tutor.")[0].strip('\n').split('the question topic is the following:')[-1].strip())

df['input_features'] = df['input_features'].apply(lambda x: preprocess_text(x))
df['input_features'] = df["llmMisconception_clean"] + "\n\n" + df['input_features']

embedding_query = model.encode(df['input_features'], convert_to_tensor=True)
embedding_Misconception = model.encode(df_misconception_mapping.MisconceptionName.values, convert_to_tensor=True)
top25 = util.semantic_search(embedding_query, embedding_Misconception, top_k=25)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [24]:
# space separated string
df["MisconceptionId"] = [" ".join([str(x["corpus_id"]) for x in top25]) for top25 in top25]

df[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)
df.head()

,QuestionId_Answer,prompt_text,llmMisconception,QuestionId,llmMisconception_clean,input_features,MisconceptionId
0,1869_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...,The misconception that leads to the incorrect ...,2488 1941 1672 987 2518 2532 1999 1005 706 150...
1,1869_C,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...,The misconception that leads to the incorrect ...,2488 1941 1672 2518 987 2532 1999 1005 1516 32...
2,1869_D,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1869,The misconception that leads to the incorrect ...,The misconception that leads to the incorrect ...,2488 2532 2518 987 1672 1941 1999 77 1005 1119...
3,1870_A,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1870,The misconception that leads to the incorrect ...,The misconception that leads to the incorrect ...,59 1540 113 885 167 683 3 2398 1593 1554 1610 ...
4,1870_B,"<|im_start|>system\nYou are Qwen, created by A...",The misconception that leads to the incorrect ...,1870,The misconception that leads to the incorrect ...,The misconception that leads to the incorrect ...,59 1540 113 885 167 683 3 2398 1554 1593 1610 ...


In [25]:
df_train

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,1700,321,Divide two decimals with the same number of de...,224,Multiplying and Dividing with Decimals,D,\( 0.9 \div 0.3= \),\( 0.3 \),\( 9.3 \),\( 0.333 \ldots \),\( 3 \),153.0,-1.0,2359.0,-1.0
